## Adapt existing .Vtu to the new format

#### Step 0: load a .vtu file as XML 

In [84]:
import xml.etree.ElementTree as ET
import numpy as np

In [85]:
#vtu_file = ("/mnt/c/Users/yoavr/Desktop/paraview_pipeline"
#            "/rework/alpha/13_balance_flip/flip_flip/timestep_000001.vtu")
vtu_file = ("/mnt/c/Users/yoavr/Desktop/paraview_pipeline/hello_chemfarm"
            "/FW_block_aggregated/timesteps/f6w14/timestep_000149.vtu"
            "")
tree = ET.parse(vtu_file)
root = tree.getroot()

In [86]:
root[3][0][0].tag

'PointData'

In [87]:
# display the structure of the xml tree
for child in root:
    print(child.tag, child.attrib)
print(r'###')
pdata = root.findall('.//PointData')[0]
pdata.tag, pdata.attrib
is_typed = False
for child in pdata:
    print(child.tag, child.attrib)
    if child.attrib['Name']=='type':
        is_typed=True
if !is_typed:
    print(".vtu file has no type! will not work correctly in the simulation")

dumpdate {}
tape {}
trisurf {'nvtx': '4502', 'npoly': '0', 'nmono': '0', 'compressed': 'false', 'seed': '1618649958'}
UnstructuredGrid {}
###
DataArray {'type': 'Int64', 'Name': 'vertices_idx', 'format': 'ascii'}
DataArray {'type': 'Float64', 'Name': 'spontaneous_curvature', 'format': 'ascii'}
DataArray {'type': 'Float64', 'Name': 'bending_energy', 'format': 'ascii'}


False

#### Step 1: reconstruct type. Determied by spontaneous curvature

In [32]:
spon_curv = pdata.findall('.//*[@Name="spontaneous_curvature"]')[0]
spon_curv.attrib

{'type': 'Float64', 'Name': 'spontaneous_curvature', 'format': 'ascii'}

In [33]:
# type 4: bare vertex. type 39: active vertex
def type_from_curvature(curvatures):
    type_array = np.zeros(curvatures.shape)
    type_array[curvatures>0]=39
    type_array[curvatures<=0]=4
    return type_array

In [34]:
#an internal function from ts_vtu_python.py: 
def _xml_to_str_list(node):
    r"""Take xml node, extract text array to list.

    Convert sequence to 1d array and lines to 2d arrays, e.g
     2 3\n4 5 6\n' -> [['2','3'],['4','5','6']]
    '\n 2 3 5' -> ['2','3','5']
    """
    all_text = node.text.strip()

    # if there are tabs in the middle, assume this is a 2d list
    if all_text.__contains__('\n'):
        return [x.split() for x in all_text.split('\n')]
    else:  # no tabs: 1d array
        return all_text.split()

curvatures_txt = _xml_to_str_list(spon_curv)

In [35]:
curvatures = list(map(float,curvatures_txt)) # convert to float
cs = np.array(curvatures)
print(cs[:10],"...")

[0.  0.  0.  0.  0.  0.  0.  0.  0.8 0. ] ...


In [36]:
ar_type = type_from_curvature(cs)
print(ar_type[:10],"...")

[ 4.  4.  4.  4.  4.  4.  4.  4. 39.  4.] ...


In [37]:
def txt_from_array(a):
    return ''.join(f'{n:.17e} ' for n in a)

In [38]:
type_txt = txt_from_array(ar_type)

In [39]:
from_c, to_c = 500, 1500
print(f"existing text [{from_c}:{to_c}]:")
print(spon_curv.text[from_c:to_c])
print(f'\ngenerated text: [{from_c}:{to_c}]')
print(type_txt[from_c:to_c]) # peek at data

existing text [500:1500]:
+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 8.00000000000000044e-01 0.00000000000000000e+00 0.00000000000000000e+00 8.00000000000000044e-01 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 8.00000000000000044e-01 0.00000000000000000e+00 0.00000000000000000e+00 0.00000000000000000e+00 8.00000000000000044e-01 0.00000000

In [82]:
# instead of updating the curvature like the previous cases, we want to add a node
new_node = ET.SubElement(pdata,"DataArray")
new_node.attrib = {"type": "Int64", "Name": "type", "format": "ascii"}

In [83]:
for child in pdata: print(child.attrib)

{'type': 'Int64', 'Name': 'vertices_idx', 'format': 'ascii'}
{'type': 'Float64', 'Name': 'spontaneous_curvature', 'format': 'ascii'}
{'type': 'Float64', 'Name': 'bending_energy', 'format': 'ascii'}
{'type': 'Int64', 'Name': 'type', 'format': 'ascii'}


3. Now we need to reconstruct the rest of the information based on the type

{'type': 'Int64', 'Name': 'vertices_idx', 'format': 'ascii'}

In [41]:
#see that the change applied
root2=tree.getroot()
print(root2.findall('.//*[@Name="spontaneous_curvature"]')[0].text.split(' ')[10:30])

['8.00000000000000044e-01', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '8.00000000000000044e-01', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '0.00000000000000000e+00', '8.00000000000000044e-01', '0.00000000000000000e+00', '0.00000000000000000e+00', '8.00000000000000044e-01']


In [42]:
#!!! uncomment to modify the vtu file!!!
#tree.write(r'/mnt/c/Users/yoavr/Desktop/paraview_pipeline/tests/round_100_neg.vtu',encoding=r'UTF-8')

<Element 'type' at 0x7f570079cb80>